In [0]:
import requests
import pickle
import gzip
from io import BytesIO
import urllib.request
import json
import numpy as np
import os
from bs4 import BeautifulSoup
!pip install ipython-autotime
!pip install jsonlines
import jsonlines
%load_ext autotime

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
time: 1min 36s


/content/drive/My Drive/DataScience/Kickstarter/downloader/data url/

In [0]:
#ALTERNATIVE 2
class file_merger(object):
  def __init__(self):
    self.url_source = 'https://webrobots.io/kickstarter-datasets/'
    self.data_destination = '/content/drive/My Drive/Data Science Projects/Kickstarter/downloader/data url/collated.json'
    self.progress_destination = '/content/drive/My Drive/Data Science Projects/Kickstarter/downloader/data url/url_prog.pkl'
    self.url_progress = {}
    self.project_data = []
    self.months = ['04','08','12'] 
    self.project_id_list = []

  def save_progress(self):
    with open(self.progress_destination,'wb') as pkl:
      pickle.dump((self.url_progress,self.project_id_list),pkl)

  def load(self):
    with open(self.progress_destination,'rb') as pkl:
        prog = pickle.load(pkl)
        self.url_progress = prog[0]
        self.project_id_list = prog[1]
  
  def get_urls(self):
    soup = BeautifulSoup(requests.get(self.url_source).content)
    years = soup.find('div',{'class':'fusion-text'}).findAll('ul')
    urls = []
    for year in years:
      try:
        if int(year.find('li').text[:4]) >= 2016:
          table = year.findAll('a')
          for link in table:
            if link.text == 'JSON':
              href = link.get('href')
              if href[68:70] in self.months:
                urls.append(href)
      except:
        pass
    self.url_progress = {x:False for x in urls}

  def download_json_gz(self,url):
    """streams compresse jason file from url"""
    print(f'Commencing download: {url}')
    r = requests.get(url, allow_redirects=True, stream=True).content
    print('File downloaded')
    return BytesIO(r)

  def extract_projects_from_json_gz(self,json_gz_file,state = ['successful','failed']):
    print('Starting to extract')
    """unzips file and appends to project_data any failed/successful projects that haven't being previously added"""
  
    with gzip.GzipFile(fileobj=json_gz_file) as fh:
      print('Unzipped')
      new_projects = []
      added = 0
      for n,item in enumerate(jsonlines.Reader(fh)):
        if item['data']['state'] not in state:
          url_p = item['data']['urls']['web']['project'].split('?')[0]
          if url_p not in self.project_id_list:
            new_projects.append(item['data'])
            self.project_id_list.append(url_p)
            added+= 1
        if n%1000 ==0:
          print(f'{n} analysed - {added} projects added')
          added = 0
        if n%20000 == 0:
          with jsonlines.open(self.data_destination, mode='a') as writer:
            writer.write_all(new_projects)
          new_projects = []
          self.save_progress()
      with jsonlines.open(self.data_destination, mode='a') as writer:
            writer.write_all(new_projects)
      self.save_progress()

  def download_process_data(self,n_files = -1):
    url_list = [url for url in self.url_progress.keys() if self.url_progress[url] == False]
    if n_files>0:
      url_list = url_list[:n_files]
    for url in url_list: 
      print('')
      gz_file = self.download_json_gz(url)
      self.extract_projects_from_json_gz(gz_file)
      self.url_progress[url] = True
      print('Completed')


In [0]:
merger = file_merger()

In [3]:
merger.load()

FileNotFoundError: ignored

In [4]:
merger.get_urls()

NameError: ignored

In [0]:
merger.url_progress

{'https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2016-04-15T02_09_04_328Z.json.gz': False,
 'https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2016-08-15T02_04_03_829Z.json.gz': False,
 'https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2016-12-15T22_20_52_411Z.json.gz': False,
 'https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2017-04-15T22_21_18_122Z.json.gz': False,
 'https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2017-08-15T22_20_51_958Z.json.gz': False,
 'https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2017-12-15T10_20_51_610Z.json.gz': False,
 'https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2018-04-12T03_20_13_192Z.json.gz': False,
 'https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2018-08-16T03_20_13_856Z.json.gz': False,
 'https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2018-12-13T03_20_05_701Z.json.gz': False,
 'https://s3.amazonaws.com/weruns/for

time: 5.34 ms


In [0]:
merger.download_process_data()


Commencing download: https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2019-12-12T03_20_05_306Z.json.gz
File downloaded
Starting to extract
Unzipped
0 analysed - 0 projects added
1000 analysed - 0 projects added
2000 analysed - 0 projects added
3000 analysed - 0 projects added
4000 analysed - 0 projects added
5000 analysed - 0 projects added
6000 analysed - 0 projects added
7000 analysed - 0 projects added
8000 analysed - 0 projects added
9000 analysed - 0 projects added
10000 analysed - 0 projects added
11000 analysed - 0 projects added
12000 analysed - 0 projects added
13000 analysed - 0 projects added
14000 analysed - 0 projects added
15000 analysed - 0 projects added
16000 analysed - 0 projects added
17000 analysed - 0 projects added
18000 analysed - 0 projects added
19000 analysed - 0 projects added
20000 analysed - 0 projects added
21000 analysed - 0 projects added
22000 analysed - 0 projects added
23000 analysed - 0 projects added
24000 analysed - 0 projects added


# Compressing File

In [0]:
file = '/content/drive/My Drive/Data Science Projects/Kickstarter/downloader/data url/collated.json'
fp = open(file,"r")
data = fp.read()
bindata = bytearray(data)
with gzip.open(file+'.gz', "wb") as f:
	f.write(bindata)

In [0]:
file = '/content/drive/My Drive/Data Science Projects/Kickstarter/downloader/data url/collated.json'
with gzip.open(file+'.gz', "rb") as f:
  data = f.read()
  print(data[:100])

b''
time: 12.4 ms


- Check that downloaded file can be read
- Deduplicate
- Create save progress function
- ensure that we can load previous ids to list_of_ids

In [0]:
pip install json-lines


time: 3.64 s


In [0]:
import ujson

time: 4.76 ms


In [0]:
import json_lines

time: 1.01 ms


In [0]:
#cleaning data
print('Commencing cleaning.')
with gzip.GzipFile('/content/drive/My Drive/DataScience/Kickstarter/collated/data/collated.json.gz','r') as file:
  lines = json_lines.reader(file)
  print(f'345467 projects to be analysed')
  new_file = b''
  ids = []
  for n,project in enumerate(lines):
    if project['profile']['id'] not in ids:
      temp = ''
      temp = json.dumps(project)+'\n'
      new_file += temp.encode('UTF-8')
      ids.append(project['profile']['id'])
    else:
      print('binned')
    if n%500 == 0:
      print(f'{n} projects analysed')

with gzip.open('/content/drive/My Drive/DataScience/Kickstarter/collated/data/collated2.json.gz', 'w') as final:
  final.write(new_file)



Commencing cleaning.
345467 projects to be analysed
0 projects analysed
500 projects analysed
1000 projects analysed
1500 projects analysed
2000 projects analysed
2500 projects analysed
3000 projects analysed
3500 projects analysed
4000 projects analysed
4500 projects analysed
5000 projects analysed
5500 projects analysed
6000 projects analysed
6500 projects analysed
7000 projects analysed
7500 projects analysed
8000 projects analysed
8500 projects analysed
9000 projects analysed
9500 projects analysed
10000 projects analysed
10500 projects analysed
11000 projects analysed
11500 projects analysed
12000 projects analysed
12500 projects analysed
13000 projects analysed
13500 projects analysed
14000 projects analysed
14500 projects analysed
15000 projects analysed
15500 projects analysed
16000 projects analysed
16500 projects analysed
17000 projects analysed
17500 projects analysed
18000 projects analysed
18500 projects analysed
19000 projects analysed
19500 projects analysed
20000 projec

KeyboardInterrupt: ignored

time: 30min 48s


In [0]:
new = 'https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2016-04-15T02_09_04_328Z.json.gz'

time: 749 µs


In [0]:
r = requests.get(new, allow_redirects=True, stream=True).content
data_bytes =  BytesIO(r)
with gzip.GzipFile(fileobj=data_bytes) as fh:
  json_data = []
  for n,item in enumerate(json_lines.reader(fh)):
        if n == 0:
          print(item)
  

{'id': 46950900, 'robot_id': 'Kickstarter', 'run_id': 'Kickstarter_2016-04-15T02_09_04_328Z', 'created_at': '2016-04-15T04:54:26.979778', 'table_id': 'Kickstarter', 'data': {'id': 1947024607, 'goal': 17000, 'name': 'Flourishing Urn (Canceled)', 'slug': 'flourishing-urn', 'urls': {'web': {'project': 'https://www.kickstarter.com/projects/490659404/flourishing-urn?ref=category', 'rewards': 'https://www.kickstarter.com/projects/490659404/flourishing-urn/rewards'}}, 'blurb': 'A Patented Cremation Urn and Pot for People and Pets. The Urn sets into the cavity in the pot, The Pot can hold live plants and flowers', 'photo': {'ed': 'https://ksr-ugc.imgix.net/projects/1668852/photo-original.jpg?w=338&h=190&fit=fill&bg=000000&v=1424021858&auto=format&q=92&s=98904cca210a1d9b5afdde7cbcff6a02', 'key': 'projects/1668852/photo-original.jpg', 'med': 'https://ksr-ugc.imgix.net/projects/1668852/photo-original.jpg?w=266&h=150&fit=fill&bg=000000&v=1424021858&auto=format&q=92&s=ea64918dd3a9e7ca1871893b5867ac